# San Francisco, CA Building Permit Mapping

## About Me
- __Author__: Marcus Collins
- __Date__: 2024-Sep-19

## Credits
* __Data Source__: [San Francisco Building Permits OpenGov database](https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9/data)

### Documentation URLs
* __Python 3__: [Python 3 Docs](https://docs.python.org/3/)
* __NumPy__: [NumPy API Reference](https://numpy.org/doc/stable/reference/index.html#reference)
* __Pandas__: [Pandas API Reference](https://pandas.pydata.org/docs/reference/index.html#api)
* __Folium__: [Folium API Reference](https://python-visualization.github.io/folium/latest/reference.html)
* __IPython__: [IPython API Reference](https://ipython.readthedocs.io/en/stable/api/index.html)

## Introduction
This is meant to demonstrate mapping within Python specifically using Pandas, IPython, NumPy and Folium.
<span id="intro"></span>

## Code

### Imports

In [1]:
## standard Python included imports
import pathlib
import re

## specially installed Python imports
try:
    import numpy as np
    import pandas as pd
    import folium as FMap
    from folium import plugins
    import IPython
    from IPython.display import display
except(ImportError) as e:
    print(f"Error: {e.args}")

### Pathnames and Filenames

In [2]:
imgpath = pathlib.Path("images")
htmlpath = pathlib.Path("html")
datapath = pathlib.Path("data")

In [3]:
permitcsvfile = datapath / "SanFranBuildingPermits_2023Jan-2023Dec.csv"

### Basic Map

In [4]:
## set upper, lower bounds (latitudes {North,South}, longitudes {East,West}) for limiting map
lat_min, lat_max = 37.54, 37.85
lon_min, lon_max = -122.54, -122.33
sfbasic = FMap.Map(
    location = (37.75429983275193, -122.44652683825896), 
    tiles = "OpenStreetMap",
    max_bound = True,
    min_lat = lat_min,
    max_lat = lat_max,
    min_lon = lon_min,
    max_lon = lon_max,
    zoom_start = 12
)
#sfbasic.save("html/SanFranBasicMap.html")

In [69]:
sfbasic

In [ ]:
basicmapfile = imgpath / "SanFranBasicMap.png"
IPython.display.Image(basicmapfile, format="png", embed=True)

[Into - Top](#top)

#### __Previewing Data Before Sampling__

In [6]:
# open full CSV file to get relevant grouping information
permit_df = pd.read_csv(
    permitcsvfile,
    header = 0,            # row(0) has the column names
    on_bad_lines = "warn", 
    memory_map = True      # use local RAM instead of constantly using I/O streams
)

In [7]:
display( permit_df.info() )
print("")
display( permit_df.head(n=15) )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24895 entries, 0 to 24894
Data columns (total 51 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Permit Number                           24895 non-null  object 
 1   Permit Type                             24895 non-null  int64  
 2   Permit Type Definition                  24895 non-null  object 
 3   Permit Creation Date                    24895 non-null  object 
 4   Block                                   24895 non-null  object 
 5   Lot                                     24895 non-null  object 
 6   Street Number                           24895 non-null  int64  
 7   Street Number Suffix                    396 non-null    object 
 8   Street Name                             24895 non-null  object 
 9   Street Suffix                           24562 non-null  object 
 10  Unit                                    3327 non-null   fl

None

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,ADU,Primary Address Flag,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Point Source,Record ID,data_as_of,data_loaded_at
0,202301039642,8,otc alterations permit,01/03/2023,3651,035,119,NaN,Vicksburg,St,...,N,Y,8.0,Noe Valley,94114.0,POINT (-122.428358089 37.752937329),eas_address_point,1659588164045,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
1,202301039602,8,otc alterations permit,01/03/2023,0225,017,856,NaN,Stockton,St,...,N,Y,3.0,Chinatown,94108.0,POINT (-122.407641249 37.793909713),eas_address_point,1659542233076,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
2,202301039614,4,sign - erect,01/03/2023,0313,003,55,NaN,Grant,Av,...,N,Y,3.0,Financial District/South Beach,94108.0,POINT (-122.405249168 37.787445119),eas_address_point,1659557357941,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
3,202301039611,8,otc alterations permit,01/03/2023,1239,026,242,NaN,Divisadero,St,...,N,Y,5.0,Haight Ashbury,94117.0,POINT (-122.436680908 37.771119223),eas_address_point,165955393489,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
4,202301039615,8,otc alterations permit,01/03/2023,6965B,017,27,NaN,Navajo,Av,...,N,Y,11.0,Outer Mission,94112.0,POINT (-122.443736826 37.719638795),eas_address_point,1659558215514,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
5,202301039620,8,otc alterations permit,01/03/2023,6748,034,115,NaN,Cuvier,St,...,N,Y,8.0,Outer Mission,94112.0,POINT (-122.428223768 37.732974208),eas_address_point,1659563212046,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
6,202301039601,7,wall or painted sign,01/03/2023,0315,003,225,NaN,Powell,St,...,N,Y,3.0,Tenderloin,94102.0,POINT (-122.408466814 37.786750877),eas_address_point,1659541389100,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
7,202301039628,4,sign - erect,01/03/2023,0288,025,375,NaN,Bush,St,...,N,Y,3.0,Financial District/South Beach,94108.0,POINT (-122.403609674 37.790672016),eas_address_point,1659572391212,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
8,202301039600,8,otc alterations permit,01/03/2023,0145,023,20,NaN,Romolo,Pl,...,N,Y,3.0,North Beach,94133.0,POINT (-122.406214759 37.798500972),parcel_centroid,1659540232048,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM
9,202301039641,8,otc alterations permit,01/03/2023,7064,043,209,NaN,Foote,Av,...,N,Y,11.0,Outer Mission,94112.0,POINT (-122.447116113 37.714251518),eas_address_point,1659587219937,09/16/2024 11:55:00 PM,09/17/2024 03:31:39 AM


In [8]:
permit_types_df = permit_df[["Permit Type","Permit Type Definition"]].value_counts()
display( permit_types_df )

Permit Type  Permit Type Definition             
8            otc alterations permit                 22880
3            additions alterations or repairs        1465
4            sign - erect                             307
2            new construction wood frame               86
7            wall or painted sign                      73
6            demolitions                               47
1            new construction                          31
5            grade or quarry or fill or excavate        6
Name: count, dtype: int64

In [9]:
# Create a list of the permit types
permit_types_list = list(range(1,9)); display(permit_types_list)

[1, 2, 3, 4, 5, 6, 7, 8]

#### __Sample the full DataFrame__

In [10]:
# Create an array of [random] row indices
df_size = len(permit_df);  display(df_size) # original was 24895

sample_size = np.int64(1000)

### Modern way to setup a random number generator (rngen) with a seed value
###   See NumPy documentation for the various generator functions available
rngen = np.random.default_rng(1357911)

### rngen.choice(
###    Largest Value or ArrayRange, 
###    size=Number of Choices, 
###    shuffle=Rearrange between picks, 
###    replace=Choices are aviable again if picked
### )
sample_indices = pd.Index(rngen.choice(df_size, size=sample_size, shuffle=False, replace=False)); display(sample_indices[:7])

24895

Index([6298, 226, 16633, 13903, 18923, 13243, 12075], dtype='int64')

In [11]:
# Keep some columns and reset the index

### List of column names
keep_cols = ["Permit Number", "Permit Type", "Permit Type Definition", "Filed Date", "Description", "Location"]

### Make a "true" copy of the rows to be sampled rather than a view of the data (referring my new variable to the old variable's instance)
sampled_df = permit_df.loc[sample_indices, keep_cols].copy(deep=True)

### keep the old row (index) values in a new column and reset the new row numbers within the sampled DataFrame
sampled_df.reset_index(drop=False, inplace=True)

### Convert the "Filed Date" column to DateTime objects instead of strings (provides more flexibility when reformatting or calculating times, etc.)
sampled_df["Filed Date"] = sampled_df["Filed Date"].apply(lambda x: pd.to_datetime(x))

### Drop any row(s) that have an empty "Location" value
sampled_df.dropna(subset=["Location"], inplace=True)

In [12]:
display( sampled_df.info() )
print("")
display( sampled_df.head(n=5) )

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   index                   999 non-null    int64         
 1   Permit Number           999 non-null    object        
 2   Permit Type             999 non-null    int64         
 3   Permit Type Definition  999 non-null    object        
 4   Filed Date              999 non-null    datetime64[ns]
 5   Description             999 non-null    object        
 6   Location                999 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 62.4+ KB


None

,index,Permit Number,Permit Type,Permit Type Definition,Filed Date,Description,Location
0,6298,202304135626,8,otc alterations permit,2023-04-13 09:02:11,"voluntary system replacement, new lf horns to ...",POINT (-122.429764994 37.791811754)
1,226,202301069771,8,otc alterations permit,2023-01-06 11:09:38,5th floor:fire alarm t.i. : add 12 horn strobe...,POINT (-122.400427156 37.787041571)
2,16633,202308285457,8,otc alterations permit,2023-08-28 15:12:41,for complaint no. 202312605 and no. 202312541....,POINT (-122.487294773 37.781216397)
3,13903,202307242770,8,otc alterations permit,2023-07-24 08:45:18,"f/a modification on roof tower building, maint...",POINT (-122.397673997 37.791861727)
4,18923,202309287561,8,otc alterations permit,2023-09-28 07:48:36,fire alarm modification on the 23rd floor suit...,POINT (-122.401969002 37.790735824)


In [13]:
# Delete the original DataFrame to save memory
del permit_df, permit_types_df

In [14]:
# Slim down the memory usage of sampled_df and show info

### Convert "Permit Type" to an unsigned 8-bit integer
sampled_df['Permit Type'] = sampled_df['Permit Type'].astype(np.uint8)

### Convert "Permit Type Definition" to a Pandas Categorical
sampled_df['Permit Type Definition'] = pd.Categorical(sampled_df['Permit Type Definition'].values, ordered=False)

In [15]:
# Write function to be called by apply function on "Location" column to extract Latitudes and Longitudes into separate columns
def extractLatLon(locstr):
    point_pattern = '^POINT\\s*\\(([-]*[\\d]+\\.[\\d]+)\\s+(-*[\\d]+\\.[\\d]+)\\)$'
    re_controller = re.compile(point_pattern)

    #display( re_controller ) ## used for testing only
    re_match = re_controller.findall(locstr)

    #display( locstr ) ## used for testing only
    #display( f"{re_match[0][1]}, {re_match[0][0]}" )  ## used for testing only

    ## Returns a formatted string to make splitting into separate columns simpler using the Series.str.split() functionality
    return f"{re_match[0][1]},{re_match[0][0]}"

In [16]:
# Split 'Location' into separate columns and show information
sampled_df[['Latitude','Longitude']] = sampled_df['Location'].apply(extractLatLon).str.split(',', expand=True)
sampled_df['Latitude'] = sampled_df['Latitude'].astype(np.float32)
sampled_df['Longitude'] = sampled_df['Longitude'].astype(np.float32)

sampled_df.drop(columns=['Location'], inplace=True)

In [17]:
display( sampled_df.info() )
print("")
display( sampled_df.head(n=5) )
print("")
display( sampled_df.tail(n=5) )

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   index                   999 non-null    int64         
 1   Permit Number           999 non-null    object        
 2   Permit Type             999 non-null    uint8         
 3   Permit Type Definition  999 non-null    category      
 4   Filed Date              999 non-null    datetime64[ns]
 5   Description             999 non-null    object        
 6   Latitude                999 non-null    float32       
 7   Longitude               999 non-null    float32       
dtypes: category(1), datetime64[ns](1), float32(2), int64(1), object(2), uint8(1)
memory usage: 49.1+ KB


None

,index,Permit Number,Permit Type,Permit Type Definition,Filed Date,Description,Latitude,Longitude
0,6298,202304135626,8,otc alterations permit,2023-04-13 09:02:11,"voluntary system replacement, new lf horns to ...",37.791813,-122.429764
1,226,202301069771,8,otc alterations permit,2023-01-06 11:09:38,5th floor:fire alarm t.i. : add 12 horn strobe...,37.787041,-122.400429
2,16633,202308285457,8,otc alterations permit,2023-08-28 15:12:41,for complaint no. 202312605 and no. 202312541....,37.781216,-122.487297
3,13903,202307242770,8,otc alterations permit,2023-07-24 08:45:18,"f/a modification on roof tower building, maint...",37.791862,-122.397675
4,18923,202309287561,8,otc alterations permit,2023-09-28 07:48:36,fire alarm modification on the 23rd floor suit...,37.790737,-122.401970


,index,Permit Number,Permit Type,Permit Type Definition,Filed Date,Description,Latitude,Longitude
995,9938,202306019000,8,otc alterations permit,2023-06-01 09:15:39,voluntary system replacement and mandatory low...,37.746845,-122.441467
996,16784,202308295480,3,additions alterations or repairs,2023-08-29 09:11:56,replace 140 dual carbon/smoke heads w/ smoke h...,37.786896,-122.424637
997,24892,202312293292,8,otc alterations permit,2023-12-29 14:26:28,re-roofing: remove and replace roofing materia...,37.737511,-122.410751
998,24569,202312223005,8,otc alterations permit,2023-12-22 09:41:45,to obtain final inspection for work approved u...,37.778446,-122.408112
999,20025,202310138624,8,otc alterations permit,2023-10-13 10:24:23,reroofing,37.738483,-122.477196


#### __Map Samples__

> Choice of folium.Icon(colors=Values) ==> \[‘red’, ‘blue’, ‘green’, ‘purple’, ‘orange’, ‘darkred’, \
    ’lightred’, ‘beige’, ‘darkblue’, ‘darkgreen’, ‘cadetblue’, ‘darkpurple’, ‘white’, ‘pink’, ‘lightblue’, ‘lightgreen’, ‘gray’, ‘black’, ‘lightgray’\]

> If I were to choose 8: 'blue', 'purple', 'gray', 'darkpurple', 'darkgreen', 'lightblue', 'black', 'darkblue'
>> __Note__: I prefer to use RGB hex codes, because the Folium color string list is very limited. 

In [62]:
# Create the map on which to add features, pins, etc.

sfpermitmap = FMap.Map(
    location = (37.75429983275193, -122.44652683825896), 
    tiles = "OpenStreetMap",
    zoom_start = 14
)

In [63]:
# Create dictionary to control colors mapped to permit type

#### Color list is in the order to be associated with each permit type 1,2,...,8
color_list = ['#269A92', '#606060', '#660505', '#CC6600', '#B0B05E', '#3B620F', '#FF99CC', '#4C74B7']

In [64]:
# Create Folium CircleMarkers with grouping, colors, tooltips and popups

### Create 8 map groups
permit_dct = dict()

for group, color in enumerate(color_list, start=1):
    if group not in permit_dct:
        permit_dct[group] = {
            "color":color,
            "grpobj": FMap.FeatureGroup(f"Permit Type {group}").add_to(sfpermitmap)
        }

display( permit_dct )

{1: {'color': '#269A92',
  'grpobj': <folium.map.FeatureGroup at 0x76121c86b410>},
 2: {'color': '#606060',
  'grpobj': <folium.map.FeatureGroup at 0x76121c86b4d0>},
 3: {'color': '#660505',
  'grpobj': <folium.map.FeatureGroup at 0x76121de462a0>},
 4: {'color': '#CC6600',
  'grpobj': <folium.map.FeatureGroup at 0x7612240ff020>},
 5: {'color': '#B0B05E',
  'grpobj': <folium.map.FeatureGroup at 0x76121ff64230>},
 6: {'color': '#3B620F',
  'grpobj': <folium.map.FeatureGroup at 0x7612240fef30>},
 7: {'color': '#FF99CC',
  'grpobj': <folium.map.FeatureGroup at 0x7612267a9d60>},
 8: {'color': '#4C74B7',
  'grpobj': <folium.map.FeatureGroup at 0x76121de45be0>}}

In [65]:
# Iterate over the DataFrame rows and create a Folium Icon within the appropriate group

#### Map Circles in Folium have their radius unit in meters instead of pixels like CircleMarkers
####  Additionally, Circles are fixed position & size while CircleMarkers change size based on zoom level
circ_radius = float(30)
circ_opacity = float(0.45)
border_weight = float(1) 


for row_tuple in sampled_df.itertuples():
    ##print( f"PermitID={row_tuple._2}, PermitType={row_tuple._3}, PermitTypeLabel={row_tuple._4}, Latitude={row_tuple.Latitude}, Longitude={row_tuple.Longitude}" )
    if row_tuple._3 in permit_dct:
        FMap.Circle(
            location = (row_tuple.Latitude,row_tuple.Longitude),
            fill = True,
            radius = circ_radius,
            color = permit_dct[row_tuple._3]["color"],
            weight = border_weight,
            fill_color = permit_dct[row_tuple._3]["color"],
            fill_opacity = circ_opacity,
            tooltip = f"Permit ID {row_tuple._2}",
            popup = FMap.Popup(
                        html = "<div><b>Permit Number</b>: {}<br/><b>Type</b>: {}<br/><b>Type Label</b>: {}<br/><b>Filed</b>: {}<br/><b>Desc</b>: {}</div>".format(row_tuple._2, row_tuple._3, row_tuple._4, row_tuple._5.strftime('%Y-%b-%d %X'), row_tuple.Description),
                        max_width = int(300)
                    )
        ).add_to(permit_dct[row_tuple._3]["grpobj"])

FMap.plugins.MousePosition(position="bottomleft").add_to(sfpermitmap)
FMap.LayerControl().add_to(sfpermitmap)

#### __Display Map with Markers__

In [66]:
sfpermitmap

#### Displaying screenshots for PDF since images only appear via [__nbconvert__](https://nbviewer.org) preview website otherwise

In [ ]:
basic_pin_image = imgpath / "SF_Jupyter_FullMap-Legend.png";
IPython.display.Image(basic_pin_image, format="png", embed=True)

In [ ]:
tooltip_pin_image = imgpath / "SF_Jupyter_FullMap-Tooltip.png";
IPython.display.Image(tooltip_pin_image, format="png", embed=True)

Go to <b><a href="#intro">Top</a></b>